In [70]:
import os
import pandas as pd
import glob
from tqdm import tqdm_notebook as tqdm
import xml.etree.ElementTree as ET
import sys
sys.path.append('../scripts/')
import util
import parsing_util

In [71]:
from importlib import reload

In [72]:
reload(parsing_util)

<module 'parsing_util' from '../scripts\\parsing_util.py'>

In [7]:
annotation_frames = '../models/metoo_icwsm2019/data/annotated_connotation_frames/'

In [9]:
os.listdir(annotation_frames)

['agency_power.csv',
 'full_frame_info.txt',
 'README_agency_power.md',
 'readme_annotation_frames.txt']

In [16]:
annotations_df = pd.read_csv(os.path.join(annotation_frames, 'full_frame_info.txt'), sep='\t')

In [17]:
annotations_df.head()

,verb,Perspective(wo),Perspective(ws),Perspective(so),Effect(o),Effect(s),Value(o),Value(s),State(o),State(s),Perspective(ro),Perspective(rs),Perspective(os)
0,have,0.366667,0.333333,0.466667,0.066667,0.200000,0.466667,0.600000,0.066667,0.366667,0.400000,0.366667,0.066667
1,say,0.000000,0.166667,0.066667,0.133333,0.066667,0.066667,1.000000,0.000000,0.100000,0.033333,0.266667,0.000000
2,make,-0.066667,0.100000,0.100000,0.466667,-0.066667,0.200000,1.000000,0.000000,0.166667,-0.033333,0.200000,0.000000
3,take,0.500000,0.600000,0.800000,0.133333,0.600000,0.466667,0.600000,0.033333,0.666667,0.500000,0.566667,0.066667
4,go,0.066667,0.133333,0.200000,0.000000,0.066667,-0.066667,0.466667,-0.100000,0.033333,0.066667,0.066667,-0.100000


In [25]:
# os.listdir('../data/news-article-flatlist/stanford-parses/')

In [73]:
stanford_files = glob.glob('../data/news-article-flatlist/stanford-parses/*')
source_verbs_output = []

for stanford_parse in tqdm( stanford_files, total=len(stanford_files)):
    fn = os.path.basename(stanford_parse).replace('.txt.xml', '')
    data = open(stanford_parse, encoding='utf-8').read()
    doc_etree = ET.fromstring(data)
    
    ## parse
    sents_list, named_ents_list, lemmas, pos_tags, deps = parsing_util.parse_etree(doc_etree)
    stanford_corefs = parsing_util.convert_coref(doc_etree=doc_etree, sentences=sents_list)
    
    ## group
    named_entity_clusters = parsing_util.extract_and_merge_ners(named_ents_list, sents_list, stanford_corefs)
    named_entity_clusters = parsing_util.extract_quotes_for_each_ner(named_entity_clusters, lemmas, pos_tags, deps)
    
    ## get all verbs associated with sources
    for source_name, mentions in list(filter(lambda x: len(x[1]['speaking_vbs']) > 0, named_entity_clusters.items())):
        for vb in mentions['all_vbs']:
            vb_lemm = lemmas[vb['sentence']][vb['verb_idx']]
            source_verbs_output.append({
                "source": source_name,
                "doc_id": fn,
                "verb_lemma": vb_lemm
            })

In [57]:
source_verbs_output_df = pd.DataFrame(source_verbs_output)

In [63]:
source_verbs_output_df['verb_lemma'].value_counts().loc[lambda s: s>3]

say              22846
have              1457
tell              1094
add                679
make               657
call               573
take               552
do                 545
want               506
ask                474
go                 470
know               456
think              444
see                365
try                355
get                336
write              331
give               330
leave              309
speak              307
be                 301
meet               294
believe            291
come               286
become             284
find               276
seem               276
describe           270
begin              268
recall             252
                 ...  
expert               4
mad                  4
overcome             4
justify              4
foreigner            4
prepared             4
71                   4
impressed            4
overrule             4
jewish               4
strip                4
exception            4
bloom      